## Libraries and Environment variables

In [ ]:
import numpy as np
import plotly.graph_objects as go
import tqdm
from scipy import stats
import plotly.figure_factory as ff
from tabulate import tabulate
WQU_COLORS = ['#3F365F','#EF8B2A', '#33EEEB']

## Some Utility Methods and Functions

In [ ]:
printTable = lambda object: tabulate([[name, getattr(object, name)] for name in dir(object) if not name.startswith('_')])
def format_plotly_figure(func):
    def wrapper (*args, **kwargs):
        fig = func(*args, **kwargs)
        fig.update_layout(
        margin=dict(l=20, r=20, t=20, b=20),
        width=900,
        height=400,
        
    )
        fig.show()
    return wrapper

@format_plotly_figure
def visualise_distribution (array_of_arrays, label):
    if not hasattr(array_of_arrays[0], "__len__"):
        array_of_arrays = [array_of_arrays] 
    fig = go.Figure()
    fig = ff.create_distplot(array_of_arrays, [label], colors = WQU_COLORS, )
    for array in array_of_arrays: print(printTable(stats.describe(array)))
    return fig

@format_plotly_figure
def line_chart (array_of_arrays):
    if not hasattr(array_of_arrays[0], "__len__"):
        array_of_arrays = [array_of_arrays]
    fig = go.Figure()
    for array in array_of_arrays:
        fig.add_trace(go.Scatter(x=np.arange(len(array)), y=array))
    return fig


#### Question 5


In [ ]:
v0 = 0.032
kappa_v = 1.85
sigma_v = 0.3
theta_v = 0.045
rho = -0.3

spot_price = 80 # Current underlying asset price
risk_free_rate = 0.055  # Risk-free rate
M0 = 1  # Number of time steps in a year
T = 3 # Number of years
M = int(M0 * T)  # Total time steps
number_of_iterations = 10000  # Number of simulations
dt = T / M  # Length of time step

In [ ]:
def generate_heston_paths(spot_price, r, v, row, cho_matrix):
    S = np.zeros((M + 1, number_of_iterations), dtype=float)
    S[0] = spot_price
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)

    return S

def generate_volatility_paths(v0, kappa, theta, sigma, T, M, number_of_iterations, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, number_of_iterations), dtype=np.float64)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

def random_number_generator(M, number_of_iterations):
    rand = np.random.standard_normal((2, M + 1, number_of_iterations))
    return rand

In [ ]:
# Generating random numbers from standard normal
rand = random_number_generator(M, number_of_iterations)
# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=np.float64)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

In [ ]:
def option_payoff(prices, strike_prices, option_type):
    if not  option_type in ['call', 'put']:
        raise ValueError('option_type must be either call or put')
    if option_type == 'call':
        payoff = prices - strike_prices
    else:
        payoff = strike_prices - prices
    payoff[payoff < 0] = 0
    return payoff

We have multiple walks of call-option payoffs each generated through a simulation of the underlying stock price. The underlying asset price is generated through simulation of volatility. Since it is an European option that is exercised at maturity, we will discount the terminal payoff to the present value. 

In [ ]:
# Volatility process paths
def price_options (v0, kappa_v, theta_v, sigma_v, T, M, number_of_iterations, rand, cho_matrix, spot_price, risk_free_rate, strike_price):
    volatility_paths = generate_volatility_paths(v0, kappa_v, theta_v, sigma_v, T, M, number_of_iterations, rand, 1, cho_matrix)
    # Underlying price process paths
    asset_prices = generate_heston_paths(spot_price, risk_free_rate, volatility_paths, 0, cho_matrix)
    asset_prices = asset_prices.T
    strike_prices = np.full(asset_prices[:,-1].shape, strike_price)
    european_call_prices= np.exp(-risk_free_rate*T)*option_payoff(asset_prices[:,-1], strike_prices, 'call')
    european_put_prices = np.exp(-risk_free_rate*T)*option_payoff(asset_prices[:,-1], strike_prices, 'put')
    european_call_price = round(np.mean(european_call_prices),2)
    european_put_price = round(np.mean(european_put_prices),2)
    print(f"The price of the European call option is: {european_call_price}")
    print(f"The price of the European put option is: {european_put_price}")
    print(f"Put Call Parity: {european_call_price + round((np.exp(-risk_free_rate*T) * strike_price), 2)} ~= {european_put_price + spot_price}")
    return {'EU_Call': round(np.mean(european_call_prices),2), 'EU_Put': round(np.mean(european_put_prices),2), 'asset_prices': asset_prices, 'volatility_paths': volatility_paths, 'european_call_prices': european_call_prices, 'european_put_prices': european_put_prices}

In [ ]:
q5 = price_options(v0, kappa_v, theta_v, sigma_v, T, M, number_of_iterations, rand, cho_matrix, spot_price, risk_free_rate, strike_price=spot_price)

We find that the distribution of the terminal prices has a high variability (SD ~35 (almost 50% of the mean)) and is not normally distributed. This is because the terminal price is a function of the terminal volatility which is a random variable. 

In [ ]:
visualise_distribution(q5['asset_prices'][:,-1], 'Asset Prices')

In [ ]:
rho = -0.7
covariance_matrix = np.zeros((2, 2), dtype=np.float64)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

#### Question 6


In [ ]:
q6 =price_options(v0, kappa_v, theta_v, sigma_v, T, M, number_of_iterations, rand, cho_matrix, spot_price, risk_free_rate, strike_price=spot_price)

### Question 7

In [ ]:
def calculate_delta (new_spot_price, original_option_price = q5):
    option_prices = price_options(v0, kappa_v, theta_v, sigma_v, T, M, number_of_iterations, rand, cho_matrix, new_spot_price, risk_free_rate, strike_price=spot_price)
    call_delta= (option_prices['EU_Call'] - original_option_price['EU_Call'])/(new_spot_price - spot_price)
    put_delta= (option_prices['EU_Put'] - original_option_price['EU_Put'])/(new_spot_price - spot_price)
    print(f"Call Delta: {call_delta}")
    print(f"Put Delta: {put_delta}")
    return {'call_delta': call_delta, 'put_delta': put_delta}    

In [ ]:
spot_prices = list(np.arange(75, 85, 2))
deltas = [calculate_delta(price) for price in spot_prices]

In [48]:
from math import inf
call_deltas = np.array([option_delta['call_delta'] for option_delta in deltas if abs(option_delta['call_delta']) != inf])
put_deltas = np.array([option_delta['put_delta'] for option_delta in deltas if abs(option_delta['put_delta']) != inf])

In [49]:
print(f"Call Delta: {round(np.mean(call_deltas),2)}")
print(f"Put Delta: {round(np.mean(put_deltas),2)}")

Call Delta: 0.47
Put Delta: -0.35


In [50]:
change_in_spot_price = (np.append(np.array(spot_prices[1:]), inf)- np.array(spot_prices))[:-1]
change_in_call_deltas = (np.append(np.array(call_deltas[1:]), inf)- np.array(call_deltas))[:-1]
change_in_put_deltas = (np.append(np.array(put_deltas[1:]), inf)- np.array(put_deltas))[:-1]
call_gamma = change_in_call_deltas/change_in_spot_price
put_gamma = change_in_put_deltas/change_in_spot_price

In [51]:
print(f"Call Gamma: {round(np.mean(call_gamma),2)}")
print(f"Put Gamma: {round(np.mean(put_gamma),2)}")

Call Gamma: 0.01
Put Gamma: 0.01
